In [3]:
!pip install -i https://test.pypi.org/simple/ supportlib

Looking in indexes: https://test.pypi.org/simple/


In [5]:
import supportlib.gettingdata as getdata
getdata.kaggle()

Saving kaggle.json to kaggle.json


In [6]:
!kaggle datasets download -d kmader/food41

100% 5.27G/5.29G [01:55<00:01, 16.8MB/s]
100% 5.29G/5.29G [01:55<00:00, 49.1MB/s]


In [0]:
!mkdir data

In [8]:
!ls

data  food41.zip  kaggle.json  sample_data


In [9]:
%cd data

/content/data


In [0]:
getdata.zipextract("/content/food41.zip")

In [11]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras.layers import Activation,Dropout,Flatten
from keras import backend as K
import tensorflow as tf
import keras

Using TensorFlow backend.


In [0]:
!mkdir image_data

In [13]:
%cd image_data

/content/data/image_data


In [0]:
getdata.zipextract("/content/data/images.zip")

In [15]:
!ls

apple_pie	    eggs_benedict	     onion_rings
baby_back_ribs	    escargots		     oysters
baklava		    falafel		     pad_thai
beef_carpaccio	    filet_mignon	     paella
beef_tartare	    fish_and_chips	     pancakes
beet_salad	    foie_gras		     panna_cotta
beignets	    french_fries	     peking_duck
bibimbap	    french_onion_soup	     pho
bread_pudding	    french_toast	     pizza
breakfast_burrito   fried_calamari	     pork_chop
bruschetta	    fried_rice		     poutine
caesar_salad	    frozen_yogurt	     prime_rib
cannoli		    garlic_bread	     pulled_pork_sandwich
caprese_salad	    gnocchi		     ramen
carrot_cake	    greek_salad		     ravioli
ceviche		    grilled_cheese_sandwich  red_velvet_cake
cheesecake	    grilled_salmon	     risotto
cheese_plate	    guacamole		     samosa
chicken_curry	    gyoza		     sashimi
chicken_quesadilla  hamburger		     scallops
chicken_wings	    hot_and_sour_soup	     seaweed_salad
chocolate_cake	    hot_dog		     shrimp_and_grits
chocolate_mousse    huevo

In [0]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize, pyramid_reduce

In [0]:
root=os.getcwd()
img_height,img_weight=224,224
batch_size=32

In [18]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True , validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(
          root,
          target_size=(224, 224),
          batch_size=32)


Found 101000 images belonging to 101 classes.


In [21]:
resnet = applications.ResNet50(weights='imagenet',include_top=False)

x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)

predictions = Dense(101,activation='softmax')(x)
model = Model(inputs=resnet.input,outputs=predictions)


model.compile(optimizer = optimizers.Adam(lr = 0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
history = model.fit_generator(train_generator,
                              steps_per_epoch = 32,
                              epochs=10,
                              shuffle=True,
                              verbose=1
                              )
model.save('my_model.h5')
model.save_weights('my_model_weights.h5')

In [0]:
labels = dict((v,k) for k,v in labels.items())

In [0]:
import cv2
image_test=cv2.imread("/content/data/image_data/apple_pie/1005649.jpg")
image_test=cv2.resize(image_test,(224,224))
image_test = np.reshape(image_test,(1,224,224,3))
result=model.predict(image_test)
result=np.argmax(result,axis=1)
#print(labels[int(result)])